## OMS Agent with ADK

<table align="left">
    <td style="text-align: center">
        <a href="https://colab.research.google.com/github/ra2085/apigee-adk-workshop/blob/main/notebooks/oms-agent-mcp.ipynb">
          <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo\"><br> Open in Colab
        </a>
      </td>
      <td style="text-align: center">
        <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fra2085%2Fapigee-adk-workshop%2Fmain%2Fnotebooks%2Foms-agent-mcp.ipynb">
          <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
        </a>
      </td>    
      <td style="text-align: center">
        <a href="https://github.com/ra2085/apigee-adk-workshop/blob/main/notebooks/oms-agent-mcp.ipynb">
          <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
        </a>
      </td>
</table>
<br />
<br />
<br />
<br />

### Install Dependencies

In [ ]:
%pip install google-adk==1.0.0 -q
%pip install litellm -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Import Dependencies

In [ ]:
import os
import sys
from google.genai import types
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts.in_memory_artifact_service import InMemoryArtifactService # Optional
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset, SseServerParams

### Set Auth

In [ ]:
CLOUD_PROJECT = "GCP_PROJECT_ID" # @param {type:"string"}
os.environ["GOOGLE_GENAI_USE_VERTEXAI"]="TRUE"
os.environ["GOOGLE_CLOUD_PROJECT"]= CLOUD_PROJECT
os.environ["GOOGLE_CLOUD_LOCATION"]="us-central1"

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

### Get Apigee MCP Tools

In [ ]:
def get_oms_tools():
  """Gets tools from the OMS MCP Server."""
  print("Attempting to connect to OMS MCP server...")

  mcp_base_url = f"https://api-{CLOUD_PROJECT}.apiservices.dev/oms-mcp-proxy/sse"
  if not mcp_base_url:
    raise ValueError("Variable MCP_BASE_URL is not set.")

  client_id = "OMS_TOOLS_API_KEY"  # @param {type:"string"}

  if not client_id:
    raise ValueError("Environment variable OMS_TOOLS_API_KEY is not set.")

  tools = MCPToolset(
      connection_params=SseServerParams(
          url=mcp_base_url,
          headers={"x-api-key": client_id},
      )
  )
  print("MCP Customer Toolset created successfully.")
  return tools

### Initialize Agent

In [ ]:
def get_oms_agent():
    """Creates a OMS Agent"""
    tools = get_oms_tools()
    print(f"Fetched {len(tools)} OMS tools from MCP server.")
    customer_agent = Agent(
        model="gemini-2.5-pro-preview-05-06",
        name="customer_agent",
        description="Central authority for all Customer Orders",
        instruction="""
        - You are the central authority for managing the critical interface between sales and backend fulfillment systems. Your primary responsibilities include the accurate submission of new sales orders into the Order Management System (OMS) or Enterprise Resource Planning (ERP) system, and providing real-time intelligence on product availability and inventory levels. You also handle all necessary data for order creation—including customer details, ordered items, quantities, pricing, and shipping/billing addresses—while also querying product information and continuously monitoring stock to inform both pre-sales availability checks and post-order fulfillment possibilities. You also act as a crucial liaison, ensuring that orders are valid, complete, and can be fulfilled based on current or incoming inventory.
        - Use the tools at your disposal to:
            - submit a new sales order into the Order Management System (OMS) or ERP handle customer details, ordered items, quantities, pricing, and shipping/billing addresses
            - query product information and pricing determine if there's any existing stock/incoming stock that could fulfill a purchase provides real-time inventory stock level checks
            - query stock availability of specific items before or after an order is placed in the OMS
            - Provide order details to the WMS for order fulfilment
        """,
        tools=[tools]
    )
    return customer_agent

### Build Agent Runtime

In [ ]:
APP_NAME="oms_app"
USER_ID="user_fs"
SESSION_ID="user_session"
async def async_main():
  session_service = InMemorySessionService()
  artifacts_service = InMemoryArtifactService()

  session = session_service.create_session(
      app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID
  )

  query = "What's the stock availability for product P002?"
  print(f"User Query: '{query}'")
  content = types.Content(role='user', parts=[types.Part(text=query)])

  root_agent = get_oms_agent()

  runner = Runner(
      app_name='oms_app',
      agent=root_agent,
      artifact_service=artifacts_service,
      session_service=session_service,
  )

  print("Running agent...")
  events_async = runner.run_async(
      session_id=session.id, user_id=session.user_id, new_message=content
  )

  async for event in events_async:
    print(f"Event received: {event}")

  print("Cleanup complete.")

### Execute Agent

In [ ]:
await async_main()